In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

In [0]:
import numpy as np
import pickle

with open('/content/drive/My Drive/Colab Notebooks/CIFAR/data_batch_1', 'rb') as fo:
  data1 = pickle.load(fo, encoding='bytes')
with open('/content/drive/My Drive/Colab Notebooks/CIFAR/data_batch_2', 'rb') as fo:
  data2 = pickle.load(fo, encoding='bytes')
with open('/content/drive/My Drive/Colab Notebooks/CIFAR/data_batch_3', 'rb') as fo:
  data3 = pickle.load(fo, encoding='bytes')
with open('/content/drive/My Drive/Colab Notebooks/CIFAR/data_batch_4', 'rb') as fo:
  data4 = pickle.load(fo, encoding='bytes')
with open('/content/drive/My Drive/Colab Notebooks/CIFAR/data_batch_5', 'rb') as fo:
  data5 = pickle.load(fo, encoding='bytes')
with open('/content/drive/My Drive/Colab Notebooks/CIFAR/test_batch', 'rb') as fo:
  test_data = pickle.load(fo, encoding='bytes')

data1_labels = np.array(data1[b'labels'], dtype=np.int)
data2_labels = np.array(data2[b'labels'], dtype=np.int)
data3_labels = np.array(data3[b'labels'], dtype=np.int)
data4_labels = np.array(data4[b'labels'], dtype=np.int)
data5_labels = np.array(data5[b'labels'], dtype=np.int)

data1_images = data1[b'data'].reshape(-1, 32, 32, 3)
data2_images = data2[b'data'].reshape(-1, 32, 32, 3)
data3_images = data3[b'data'].reshape(-1, 32, 32, 3)
data4_images = data4[b'data'].reshape(-1, 32, 32, 3)
data5_images = data5[b'data'].reshape(-1, 32, 32, 3)


whole_labels = np.concatenate((data1_labels, data2_labels, data3_labels, data4_labels, data5_labels))
whole_images = np.concatenate((data1_images, data2_images, data3_images, data4_images, data5_images), axis=0)


test_labels = np.array(test_data[b'labels'], dtype=np.int)
test_images = test_data[b'data'].reshape(-1, 32, 32, 3)


images_train = whole_images.astype(np.float)
labels_train = whole_labels

images_test = test_images.astype(np.float)
labels_test = test_labels

print('[*] Data Is Loaded Successfully!')

[*] Data Is Loaded Successfully!


In [0]:
import tensorflow as tf


train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': images_train},
    y=labels_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True
)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': images_test},
    y=labels_test,
    num_epochs=1,
    shuffle=False
)

In [0]:
def cnn_model_fn(features, labels, mode):
  # Input Layer
    input_layer = tf.reshape(features['x'], [-1, 32, 32, 3])

    # Convolutional Layer 1
    convolution1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu
    )
    pooling1 = tf.layers.average_pooling2d(inputs=convolution1, pool_size=[2, 2], strides=2)

    # Convolutional Layer 2
    convolution2 = tf.layers.conv2d(
        inputs=pooling1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu
    )

    pooling2 = tf.layers.average_pooling2d(inputs=convolution2, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer 3
    convolution3 = tf.layers.conv2d(
        inputs=pooling2,
        filters=128,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu
    )

    pooling3 = tf.layers.average_pooling2d(inputs=convolution3, pool_size=[2, 2], strides=2)

    # Dense Layer
    pooling2_flat = tf.reshape(pooling3, [-1, 4 * 4 * 128])
    dense = tf.layers.dense(inputs=pooling2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        'classes': tf.argmax(logits, axis=1),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [0]:
classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir='.drive/My Drive/Colab Notebooks/Models/cifar3')

tensors_to_log = {'probabilities': 'softmax_tensor'}

logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)
tf.logging.set_verbosity(tf.logging.INFO)

classifier.train(input_fn=train_input_fn, steps=20000, hooks=[logging_hook])

Output hidden; open in https://colab.research.google.com to view.

In [0]:
eval_results = classifier.evaluate(input_fn=test_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-26T00:19:32Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from .drive/My Drive/Colab Notebooks/Models/cifar3/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-26-00:19:35
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.5861, global_step = 20000, loss = 1.1661602
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: .drive/My Drive/Colab Notebooks/Models/cifar3/model.ckpt-20000
{'accuracy': 0.5861, 'loss': 1.1661602, 'global_step': 20000}
